In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def extend_image(image, H):
    """
    将图像边界采用几何膨胀的方式扩展至指定大小。

    参数：
    - image: 输入图像，NumPy 数组格式。
    - desired_size: 目标尺寸，格式为 (高度, 宽度)。

    返回：
    - 扩展后的图像。
    """
    # 获取当前图像尺寸
    h, w = image.shape[:2]
    desired_h = h+2*H
    desired_w = w+H

    # 计算需要填充的像素数
    pad_top = max((desired_h - h) // 2, 0)
    pad_bottom = max(desired_h - h - pad_top, 0)
    pad_left = max((desired_w - w) // 2, 0)
    pad_right = max(desired_w - w - pad_left, 0)

    # 用零值（黑色）填充图像边界
    padded_image = cv2.copyMakeBorder(
        image, pad_top, pad_bottom, pad_left, pad_right,
        borderType=cv2.BORDER_CONSTANT, value=0
    )

    # 创建原始图像区域的掩码
    Mask = np.zeros((desired_h, desired_w), dtype=np.uint8)
    Mask[pad_top:pad_top+h, pad_left:pad_left+w] = 255

    # 复制填充后的图像
    Dilated_image = padded_image.copy()

    # 创建结构元素
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    
    for i in range(H):
        dilated_image=Dilated_image[H-(i+1):h+H+i, H-(i+1):w+H+i]
        mask= Mask[H-(i+1):h+H+i, H-(i+1):w+H+i]
        # 迭代膨胀，直到掩码区域被填充完毕
        while True:
            prev_dilated_image = dilated_image.copy()
            dilated_image = cv2.dilate(dilated_image, kernel, iterations=1)
            # 仅更新填充值的区域
            dilated_image = np.where(
                mask[..., np.newaxis] == 0, dilated_image, prev_dilated_image
            )
            # 更新掩码
            mask = cv2.dilate(mask, kernel, iterations=1)
            # 检查是否整个图像都已填充
            if np.all(mask == 255):
                break

        return dilated_image

# 读取图片
image = cv2.imread('test.png')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 调用extend_image函数进行扩展
extended_image = extend_image(image_rgb,200)

# 使用matplotlib显示原始图像和扩展图像
plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.imshow(image_rgb)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(extended_image)
plt.title('Extended Image')
plt.axis('off')

plt.show()

# 保存扩展后的图像
extended_image_bgr = cv2.cvtColor(extended_image, cv2.COLOR_RGB2BGR)
cv2.imwrite('extended_test.png', extended_image_bgr)
print("扩展后的图像已保存为 'extended_test.png'")

(1046, 1902, 3)
(1045, 1901, 3)


ValueError: could not broadcast input array from shape (1094,1950,3) into shape (1045,1901,3)